**Install python dependencies**

In [1]:
!pip install -q -r ./requirements.txt

You should consider upgrading via the '/home/jakob/dev/fm-analytics/src/notebooks/venv/bin/python3 -m pip install --upgrade pip' command.


**Load python libraries**

In [2]:
from sqlalchemy import create_engine
from os import getenv
import requests
import pandas as pd
from statistics import median
from datetime import datetime
import smart_match


**Connect to database**

In [3]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

In [4]:
ODDS_API_URL = "https://api.the-odds-api.com/v3/odds/"
ODDS_API_KEY = getenv("ODDS_API_KEY")

In [5]:
res = requests.get(ODDS_API_URL, params={
    'api_key': ODDS_API_KEY,
    'region': 'eu',
    'sport': 'soccer_germany_bundesliga'
})

odds = res.json()

In [6]:
column_names = ["club", "odd_win", "odd_lose", "odd_draw"]
df_odds = pd.DataFrame(columns = column_names)

matches = odds['data']

for match in matches:
    club_1 = match['teams'][0]
    club_2 = match['teams'][1]

    odds_club1, odds_club2, odds_draw = [], [], []

    betting_sites = match['sites']

    for site in betting_sites:
        # key can be h2h or h2h_lay
        for key in site['odds']:
             odds_club1.append(site['odds'][key][0])
             odds_club2.append(site['odds'][key][1])
             odds_draw.append(site['odds'][key][2])

    df_odds = df_odds.append({'club': club_1, 'odd_win': median(odds_club1), 'odd_lose': median(odds_club2), 'odd_draw': median(odds_draw)}, ignore_index=True)
    df_odds = df_odds.append({'club': club_2, 'odd_win': median(odds_club2), 'odd_lose': median(odds_club1), 'odd_draw': median(odds_draw)}, ignore_index=True)

df_odds

,club,odd_win,odd_lose,odd_draw


In [7]:
df_matchdays = pd.read_sql_table('matchdays', db)
current_timestamp = datetime.now(tz=None).isoformat()


upcoming_matchday_id = db.execute(f"""select id from (select min(number) as number from matchdays
where start > '2021-05-22T13:29:00Z') sub
inner join matchdays m on m.number = sub.number;""").fetchone()[0]

df_odds['matchday_id'] = upcoming_matchday_id

In [11]:
df_teams = pd.read_sql_table('teams', db)

df_odds['club_id'] = ''

for index_odd, odd in df_odds.iterrows():
    most_similar = {'db_club_id': '', 'similarity': 0 }
    for index_db, db_team in df_teams.iterrows():
        odd_name = odd['club']
        db_name = db_team['name']
        similarity = smart_match.similarity(db_name, odd_name)

        if most_similar['similarity'] < similarity:
            most_similar['db_club_id'] = db_team['id']
            most_similar['similarity'] = similarity

    odd['club_id'] = most_similar['db_club_id']
    df_odds.iloc[index_odd] = odd


df_odds = df_odds[['club_id', 'odd_win', 'odd_lose', 'odd_draw', 'matchday_id']]

# df_odds.to_sql('odds', con=db, if_exists='append')


**Old Linear Regression Code**

In [ ]:
# df_player_type = df_player_type.sort_values(by=['matchday'])


# x = np.array(df_player_type['matchday']).reshape((-1, 1))
# y = np.array(df_player_type['count'])
# model = LinearRegression().fit(x, y)
# x_new = np.array(upcoming_matchday).reshape((-1, 1))
# y_pred = model.predict(x_new)
# pred_int = y_pred[0].round(0).astype(int)